In [3]:
# %pip install tensorflow==2.4.1
# %pip install transformers
# %pip install pyarrow
# %pip install tensorflow-addons

In [1]:
import tensorflow as tf
import pandas as pd
import pickle
import os
import tensorflow_addons as tfa
from transformers import RobertaTokenizer, RobertaTokenizerFast, TFRobertaModel, TFAlbertModel

AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
model_iteration = 'iteration_1'

In [2]:
tf.config.list_physical_devices()

2021-10-15 12:44:09.648350: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [19]:
with open(f"./{model_iteration}/vocab/topics_vocab.pkl", "rb") as f:
    target_vocab = pickle.load(f)
    
with open(f"./{model_iteration}/vocab/doc_type_vocab.pkl", "rb") as f:
    doc_vocab = pickle.load(f)
    
with open(f"./{model_iteration}/vocab/journal_name_vocab.pkl", "rb") as f:
    journal_vocab = pickle.load(f)


In [20]:
encoding_layer = tf.keras.layers.experimental.preprocessing.CategoryEncoding(
    max_tokens=len(target_vocab)+1, output_mode="binary", sparse=False)

# loss_fn = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
loss_fn = tfa.losses.SigmoidFocalCrossEntropy(alpha=0.25, gamma=2.0, 
                                              reduction=tf.keras.losses.Reduction.NONE)

metric_1 = tf.keras.metrics.CategoricalAccuracy()
metric_2 = tf.keras.metrics.Recall()
metric_3 = tf.keras.metrics.Precision()
metric_4 = tf.keras.metrics.TopKCategoricalAccuracy(K=10)
# Eventually will use with focal loss


In [65]:
class CustomModel(tf.keras.Model):
    def train_step(self, inputs):
        old_features, labels = inputs
        labels = tf.RaggedTensor.from_tensor(labels, padding=0)
        paper_titles = old_features[0][:,:512].to_tensor(shape=[None, 512])
        paper_masks = old_features[1][:,:512].to_tensor(shape=[None, 512])
        
        features = (paper_titles, paper_masks, old_features[2], old_features[3])
        labels = encoding_layer(labels)

        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = loss_fn(labels, predictions)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        metric_1.update_state(labels, predictions)
        metric_2.update_state(labels, predictions)
        metric_3.update_state(labels, predictions)
        metric_4.update_state(labels, predictions)

        return {"loss": loss, 
                "accuracy": metric_1.result(), 
                "recall": metric_2.result(), 
                "precision": metric_3.result(), 
                "topK15": metric_4.result()}
  
    def test_step(self, inputs):
        old_features, labels = inputs
        labels = tf.RaggedTensor.from_tensor(labels, padding=0)
        paper_titles = old_features[0][:,:512].to_tensor(shape=[None, 512])
        paper_masks = old_features[1][:,:512].to_tensor(shape=[None, 512])
        
        features = (paper_titles, paper_masks, old_features[2], old_features[3])
        labels = encoding_layer(labels)

        with tf.GradientTape() as tape:
            predictions = self(features, training=False)
            loss = loss_fn(labels, predictions)

        metric_1.update_state(labels, predictions)
        metric_2.update_state(labels, predictions)
        metric_3.update_state(labels, predictions)
        metric_4.update_state(labels, predictions)

        return {"loss": loss, 
                "accuracy": metric_1.result(), 
                "recall": metric_2.result(), 
                "precision": metric_3.result(), 
                "topK15": metric_4.result()}
    @property
    def metrics(self):
        return [metric_1, metric_2, metric_3]

In [41]:
def _parse_function(example_proto):

    feature_description = {
        'paper_title': tf.io.RaggedFeature(tf.int64),
        'paper_mask': tf.io.RaggedFeature(tf.int64),
        'journal': tf.io.FixedLenFeature((1,), tf.int64),
        'doc_type': tf.io.FixedLenFeature((1,), tf.int64),
        'targets': tf.io.FixedLenFeature((20,), tf.int64)
    }

    example = tf.io.parse_single_example(example_proto, feature_description)

    paper_title = example['paper_title']
    paper_mask = example['paper_mask']
    doc_type = example['doc_type']
    journal = example['journal']
    targets = example['targets']

    return (paper_title, paper_mask, doc_type, journal), targets

In [42]:
def get_dataset(path, num_parts, batch_size, data_type='train'):
    
    tfrecords = [f"{path}{data_type}/{x}" for x in os.listdir(f"{path}{data_type}/") if x.endswith('tfrecord')]
    
    raw_dataset = tf.data.TFRecordDataset(tfrecords, num_parallel_reads=AUTO)

    parsed_dataset = raw_dataset.map(_parse_function, num_parallel_calls=AUTO)

    parsed_dataset = parsed_dataset \
    .apply(tf.data.experimental.dense_to_ragged_batch(256, drop_remainder=True)).shuffle(2048)
    return parsed_dataset.prefetch(AUTO)

In [46]:
file_path = f'./{model_iteration}/tfrecords/'

In [66]:
# train_ds = get_dataset(file_path, 30, 256, 'train')
val_ds = get_dataset(file_path, 1, 256, 'val')

In [40]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [68]:
with mirrored_strategy.scope():
  
    model = TFAlbertModel.from_pretrained('albert-base-v2')
    model.layers[0].trainable = False

    # Model Inputs
    paper_title_input_ids = tf.keras.layers.Input((512,), dtype=tf.int64, name='paper_title_ids')
    paper_title_att_mask = tf.keras.layers.Input((512,), dtype=tf.int64, name='paper_title_mask')
    doc_type_id = tf.keras.layers.Input((1,), dtype=tf.int64, name='doc_type_id')
    journal_id = tf.keras.layers.Input((1,), dtype=tf.int64, name='journal_id')

    # Using HF Model for Title Representation
    paper_title_embs = model(input_ids = paper_title_input_ids, 
                             attention_mask=paper_title_att_mask, 
                             output_hidden_states=True, 
                             training=False).last_hidden_state

    # Embedding Layers
    #     paper_title_embs = tf.keras.layers.Embedding(input_dim=50258, 
    #                                                  output_dim=512, 
    #                                                  mask_zero=False, 
    #                                                  trainable=True,
    #                                                  name="title_embedding")(paper_title_input_ids)

    doc_embs = tf.keras.layers.Embedding(input_dim=len(doc_vocab)+1, 
                                          output_dim=32, 
                                          mask_zero=False, 
                                          name="doc_type_embedding")(doc_type_id)

    journal_embs = tf.keras.layers.Embedding(input_dim=len(journal_vocab)+1, 
                                              output_dim=128, 
                                              mask_zero=False, 
                                              name="journal_embedding")(journal_id)

    # First layer
    dense_output = tf.keras.layers.Dense(1024, activation='relu', 
                                         kernel_regularizer='L2', name="dense_1")(paper_title_embs)
    dense_output = tf.keras.layers.Dropout(0.20, name="dropout_1")(dense_output)
    dense_output = tf.keras.layers.LayerNormalization(epsilon=1e-6, name="layer_norm_1")(dense_output)
    dense_output_flat = tf.keras.layers.GlobalAveragePooling1D(name="title_pooling_layer")(dense_output)
    doc_flat = tf.keras.layers.GlobalAveragePooling1D(name="doc_pooling_layer")(doc_embs)
    journal_flat = tf.keras.layers.GlobalAveragePooling1D(name="journal_pooling_layer")(journal_embs)
    concat_output = tf.concat(values=[dense_output_flat, journal_flat, doc_flat], axis=1)

    # Second layer
    dense_output = tf.keras.layers.Dense(1024, activation='relu', 
                                         kernel_regularizer='L2', name="dense_2")(concat_output)
    dense_output = tf.keras.layers.Dropout(0.20, name="dropout_2")(dense_output)
    dense_output = tf.keras.layers.LayerNormalization(epsilon=1e-6, name="layer_norm_2")(dense_output)

    # Third Layer
    dense_output = tf.keras.layers.Dense(256, activation='relu', 
                                         kernel_regularizer='L2', name="dense_3")(dense_output)
    dense_output = tf.keras.layers.Dropout(0.20, name="dropout_3")(dense_output)
    dense_output = tf.keras.layers.LayerNormalization(epsilon=1e-6, name="layer_norm_3")(dense_output)
    dense_output_flat = tf.keras.layers.GlobalAveragePooling1D(name="title_pooling_layer")(dense_output)


    # Output Layer
    final_output = tf.keras.layers.Dense(len(target_vocab)+1, activation="sigmoid", 
                                         name="cls")(dense_output_flat)

    test_model = CustomModel(inputs=[paper_title_input_ids, paper_title_att_mask, doc_type_id, journal_id], 
                             outputs=final_output, name='test_model')

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [69]:
test_model.compile(optimizer=optimizer)

In [70]:
test_model.summary()

Model: "test_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
paper_title_ids (InputLayer)    [(None, 512)]        0                                            
__________________________________________________________________________________________________
paper_title_mask (InputLayer)   [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_albert_model_7 (TFAlbertMode TFBaseModelOutputWit 11683584    paper_title_ids[0][0]            
                                                                 paper_title_mask[0][0]           
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512, 1024)    787456      tf_albert_model_7[0][0] 

In [71]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(f'./models/{model_iteration}/{model_iteration}_first_try', 
                                                save_best_only=False, save_weights_only=False)]

## First try (with all variables and Albert model output)

In [55]:
history = test_model.fit(train_ds, epochs=5, validation_data=val_ds, verbose=1, callbacks=callbacks)

Epoch 1/2
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
74215/74215 [==============================] - 9759s 131ms/step - loss: 6.2373 - accuracy: 0.3410 - recall: 0.0135 - precision: 0.0280 - val_loss: 4.1578 - val_accuracy: 0.4481 - val_recall: 0.0933 - val_precision: 0.5296
INFO:tensorflow:Assets written to: ./models/baseline/first_try/assets
Epoch 2/2
74215/74215 [==============================] - 9745s 131ms/step - loss: 3.8262 - accuracy: 0.4863 - recall: 0.0540 - precision: 0.7250 - val_loss: 4.6855 - val_accuracy: 0.3344 - val_recall: 0.1575 - val_precision: 0.4799
INFO:tensorflow:Assets written to: ./models/baseline/first_try/assets


## ARCHIVE: Baseline Second Try (trainable embeddings)

In [23]:
history = test_model.fit(train_ds, epochs=5, validation_data=val_ds, verbose=1, callbacks=callbacks)

Epoch 1/5
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
74215/74215 [==============================] - 12309s 165ms/step - loss: 6.1006 - accuracy: 0.3679 - recall: 0.0327 - precision: 0.0937 - val_loss: 4.5469 - val_accuracy: 0.4830 - val_recall: 0.0926 - val_precision: 0.5677
INFO:tensorflow:Assets writte